In [33]:
import json
import pandas as pd
from datetime import datetime




In [34]:
# Load the JSON data
with open('./Fit/All data/raw_com.google.step_count.delta_com.google.and.json', 'r') as file:
    data = json.load(file)

# Extract the data points
data_points = data['Data Points']



In [35]:
# Starting an empty list to store the transformed data
transformed_data = []

for point in data_points:
    steps = point['fitValue'][0]['value']['intVal']
    start_time = int(point['startTimeNanos'])
    end_time = int(point['endTimeNanos'])
    
    # Converting nanoseconds to datetime
    start_datetime = datetime.fromtimestamp(start_time / 1e9)
    end_datetime = datetime.fromtimestamp(end_time / 1e9)
    
    transformed_data.append({
        'steps': steps,
        'start_time': start_datetime,
        'end_time': end_datetime
    })

df = pd.DataFrame(transformed_data)


In [36]:
# Aggregating 
df['date'] = df['start_time'].dt.date
daily_steps = df.groupby('date')['steps'].sum().reset_index()

print(daily_steps)


          date   steps
0   2024-06-08    3615
1   2024-06-09    2491
2   2024-06-10   18051
3   2024-06-11   13049
4   2024-06-12   16118
5   2024-06-13   13301
6   2024-06-14   16142
7   2024-06-15    1950
8   2024-06-16    8883
9   2024-06-17   18681
10  2024-06-18    9249
11  2024-06-19    9895
12  2024-06-20   11682
13  2024-06-21    2262
14  2024-06-22    2939
15  2024-06-23    9574
16  2024-06-24   16989
17  2024-06-25    9963
18  2024-06-26    9079
19  2024-06-27   16190
20  2024-06-28   13895
21  2024-06-29    2641
22  2024-06-30    9679
23  2024-07-01    8469
24  2024-07-02    8051
25  2024-07-03    8659
26  2024-07-04   12646
27  2024-07-05    7947
28  2024-07-06    4552
29  2024-07-07    2105
30  2024-07-08    9542
31  2024-07-09   15188
32  2024-07-10    8000
33  2024-07-11  196007


### I will separate this data into weeks before merging with the football performance data. This will help in making the analysis more structured and easier to manage.

In [39]:
daily_steps['date'] = pd.to_datetime(daily_steps['date'])

# Subtract 4 days to shift the week start to Friday
daily_steps['adjusted_date'] = daily_steps['date'] - pd.Timedelta(days=4)

# Sort by adjusted_date to ensure weeks are calculated correctly
daily_steps = daily_steps.sort_values('adjusted_date')

# Add a 'week' column based on adjusted_date
daily_steps['week'] = daily_steps['adjusted_date'].dt.isocalendar().week

# Group by week and sum the steps
weekly_steps_sum = daily_steps.groupby('week')['steps'].sum().reset_index()

# Merge the weekly sums back into the original DataFrame
df_steps = pd.merge(daily_steps, weekly_steps_sum, on='week', suffixes=('', '_weekly'))

# Rename columns for clarity
df_steps.rename(columns={'steps_weekly': 'weekly_steps'}, inplace=True)

# Enumerate the week column starting from 1
df_steps['week'] = df_steps['week'].rank(method='dense').astype(int)

# Drop the adjusted_date column as it's no longer needed
df_steps.drop(columns=['adjusted_date'], inplace=True)

print(df_steps.tail(10))


         date   steps  week  weekly_steps
24 2024-07-02    8051     4         64040
25 2024-07-03    8659     4         64040
26 2024-07-04   12646     4         64040
27 2024-07-05    7947     5        243341
28 2024-07-06    4552     5        243341
29 2024-07-07    2105     5        243341
30 2024-07-08    9542     5        243341
31 2024-07-09   15188     5        243341
32 2024-07-10    8000     5        243341
33 2024-07-11  196007     5        243341


### I noticed some inconsistencies in the day 2024-07-11, I believe that amount of steps is wrong.
### "33  2024-07-11  196007"
### I will need to fix this and copy mannually from the app, as it's just one data


In [42]:
# Updating the steps
df_steps.loc[df_steps.index == 33, 'steps'] = 9362

# Recalculate the weekly steps for week 5
week_5_steps = df_steps[df_steps['week'] == 5]['steps'].sum()

# Update the weekly_steps column for all entries in week 5
df_steps.loc[df_steps['week'] == 5, 'weekly_steps'] = week_5_steps

print(df_steps)


         date  steps  week  weekly_steps
0  2024-06-08   3615     1         66625
1  2024-06-09   2491     1         66625
2  2024-06-10  18051     1         66625
3  2024-06-11  13049     1         66625
4  2024-06-12  16118     1         66625
5  2024-06-13  13301     1         66625
6  2024-06-14  16142     2         76482
7  2024-06-15   1950     2         76482
8  2024-06-16   8883     2         76482
9  2024-06-17  18681     2         76482
10 2024-06-18   9249     2         76482
11 2024-06-19   9895     2         76482
12 2024-06-20  11682     2         76482
13 2024-06-21   2262     3         66996
14 2024-06-22   2939     3         66996
15 2024-06-23   9574     3         66996
16 2024-06-24  16989     3         66996
17 2024-06-25   9963     3         66996
18 2024-06-26   9079     3         66996
19 2024-06-27  16190     3         66996
20 2024-06-28  13895     4         64040
21 2024-06-29   2641     4         64040
22 2024-06-30   9679     4         64040
23 2024-07-01   

# Merging the performance file into the dataframe

In [47]:

# Loading the CSV file
football_performance = pd.read_csv('Football performance.csv')

# I need to raname the column to match the existing dataframe column
football_performance.rename(columns={'Football Date': 'date'}, inplace=True)

# Convert the 'date' column to datetime to ensure it matches the 'date' format in df_steps
football_performance['date'] = pd.to_datetime(football_performance['date'])

# Merging both column by the date column
df = pd.merge(df_steps, football_performance, on='date', how='left')

# Display the merged DataFrame
print(df)


         date  steps  week  weekly_steps  Goal Scored  Goal Assisted  Team Win
0  2024-06-08   3615     1         66625          NaN            NaN       NaN
1  2024-06-09   2491     1         66625          NaN            NaN       NaN
2  2024-06-10  18051     1         66625          NaN            NaN       NaN
3  2024-06-11  13049     1         66625          NaN            NaN       NaN
4  2024-06-12  16118     1         66625          NaN            NaN       NaN
5  2024-06-13  13301     1         66625          4.0            1.0       4.0
6  2024-06-14  16142     2         76482          NaN            NaN       NaN
7  2024-06-15   1950     2         76482          NaN            NaN       NaN
8  2024-06-16   8883     2         76482          NaN            NaN       NaN
9  2024-06-17  18681     2         76482          NaN            NaN       NaN
10 2024-06-18   9249     2         76482          NaN            NaN       NaN
11 2024-06-19   9895     2         76482          Na

### I will need to replace the NaN values with 0, in order to facilitate any further visualization.
### I will also create a new column to be more clear to compare days that I played football to days that I didn't play.

In [51]:
# Fill NaN values with zeros
df.fillna(0, inplace=True)

# Create a new column to indicate football days
df['football_day'] = df['Goal Scored'] > 0

print(df.tail())

         date  steps  week  weekly_steps  Goal Scored  Goal Assisted  \
29 2024-07-07   2105     5         56696          0.0            0.0   
30 2024-07-08   9542     5         56696          0.0            0.0   
31 2024-07-09  15188     5         56696          0.0            0.0   
32 2024-07-10   8000     5         56696          0.0            0.0   
33 2024-07-11   9362     5         56696          1.0            1.0   

    Team Win  football_day  
29       0.0         False  
30       0.0         False  
31       0.0         False  
32       0.0         False  
33       3.0          True  
